In [146]:
import googlemaps

In [147]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [148]:
%aimport google_api
g = google_api.GoogleApi()
result = g.travel(origin=[38.8568095, -77.0542624],
                  destination='AMC Georgetown 14, 3111 K St NW, Washington, DC 20007', 
                  mode='transit',
                  departure_time='2019-04-13-09-30')

In [149]:
import datetime as dt
from datetime import datetime
def add_time(start_time, add_time):
    if not isinstance(start_time, datetime):  # input is not a datetime object
        if len(start_time.split('-')) != 5:
            raise ValueError('Wrong time format, should be year-month-day-hour-minute')
        year, month, day, hour, minute = start_time.split('-')
        start = dt.datetime(int(year), int(month), int(day), int(hour), int(minute))
        end = start + dt.timedelta(seconds=add_time)
    else:
        end = start_time + dt.timedelta(seconds=add_time)
    return end
add_time('2019-04-13-09-30',100)

datetime.datetime(2019, 4, 13, 9, 31, 40)

In [150]:
result

{'steps': [{'start': {'lat': 38.8568095, 'lng': -77.0542619},
   'end': {'lat': 38.8580163, 'lng': -77.05154379999999},
   'html': 'Walk to Crystal City Station',
   'travel_mode': 'WALKING'},
  {'start': {'lat': 38.8580227, 'lng': -77.0514099},
   'end': {'lat': 38.9008727, 'lng': -77.0504606},
   'html': 'Metro rail towards Largo Town Center',
   'travel_mode': 'TRANSIT'},
  {'start': {'lat': 38.90077309999999, 'lng': -77.05046089999999},
   'end': {'lat': 38.9025719, 'lng': -77.0617568},
   'html': 'Walk to 3111 K St NW, Washington, DC 20007, USA',
   'travel_mode': 'WALKING'}],
 'metro': [{'start': {'location': {'lat': 38.8580227, 'lng': -77.0514099},
    'name': 'Crystal City Station'},
   'end': {'location': {'lat': 38.9008727, 'lng': -77.0504606},
    'name': 'Foggy Bottom-GWU'},
   'duration': 660}],
 'bus': [],
 'walk': [{'duration': 267}, {'duration': 964}],
 'total_time': 1893}

In [151]:
from find_nearest_bike_location import *

In [152]:
find_bike = FindNearestBikeLocation()
depart, dest = [38.885300, -77.050000], [38.857766, -77.059580]
print(find_bike.find_neareast_bike_station(depart, dest))

([[38.8853, -77.05]], [[38.857766, -77.05958]], {'station_id': 204, 'name': 'Lincoln Memorial'}, {'station_id': 5, 'name': 'Aurora Hills Cmty Ctr / 18th St & S Hayes St'})


## 1. bicycle only

In [153]:
mode = ['walking', 'transit', 'bicycling']

In [154]:
def time_of_bicycling(depart,dest,depart_time='2019-04-13-09-30'):
    '''
    it's a problem of walking + bicycling + walking
    Args : depart --> (latitude,longitude)
           dest --> (latitude,longitude)
    Return : path, path_walk1, path_bicycle, path_walk2, total_time
    '''
    
    # bicycling location
    bike_depart,bike_dest,bike_depart_name,bike_dest_name = find_bike.find_neareast_bike_station(depart, dest)
    
    # walking
    walk_res1 = g.travel(origin=depart,
                         destination=bike_depart[0], 
                         mode='walking',
                         departure_time=depart_time)
    
    depart_time2 = add_time(depart_time,walk_res1['total_time'])
    # bicycle
    bicycle_result = g.travel(origin=bike_depart[0],
                              destination=bike_dest[0], 
                              mode='bicycling',
                              departure_time=depart_time2)
    
    depart_time3 = add_time(depart_time2,bicycle_result['total_time'])
    
    walk_res2 = g.travel(origin=bike_dest[0],
                         destination=dest, 
                         mode='walking',
                         departure_time=depart_time3)
    #bicycle path
    path_bicycle = []
    for i in result['steps']:
        tmp_depart = []
        tmp_depart.append(i['start']['lat'])
        tmp_depart.append(i['start']['lng'])

        tmp_dest = []
        tmp_dest.append(i['end']['lat'])
        tmp_dest.append(i['end']['lng'])

        path_bicycle.append([tmp_depart,tmp_dest]) 
    walk_time = walk_res1['total_time'] + walk_res2['total_time']
    
    # first walk path
    path_walk1 = []
    for i in walk_res1['steps']:
        tmp_depart = []
        tmp_depart.append(i['start']['lat'])
        tmp_depart.append(i['start']['lng'])

        tmp_dest = []
        tmp_dest.append(i['end']['lat'])
        tmp_dest.append(i['end']['lng'])

        path_walk1.append([tmp_depart,tmp_dest]) 
    # second walk path   
    path_walk2 = []
    for i in walk_res2['steps']:
        tmp_depart = []
        tmp_depart.append(i['start']['lat'])
        tmp_depart.append(i['start']['lng'])

        tmp_dest = []
        tmp_dest.append(i['end']['lat'])
        tmp_dest.append(i['end']['lng'])

        path_walk2.append([tmp_depart,tmp_dest]) 
    
    
    # return path and time
    total_time = 0
    total_time += bicycle_result['total_time'] + walk_time
    
    path = []
    path += path_walk1 + path_bicycle + path_walk2
    return path_walk1, path_bicycle, path_walk2, total_time

In [155]:
path_walk1, path_bicycle, path_walk2, total_time = time_of_bicycling(depart,dest)

In [156]:
print('path_walk1:',path_walk1)
print()
print('path_bicycle:',path_bicycle)
print()
print('path_walk2:',path_walk2)
print()
print('total_time:',total_time)
print()

path_walk1: [[[38.8860017, -77.0492223], [38.8860017, -77.0492223]]]

path_bicycle: [[[38.8568095, -77.0542619], [38.8580163, -77.05154379999999]], [[38.8580227, -77.0514099], [38.9008727, -77.0504606]], [[38.90077309999999, -77.05046089999999], [38.9025719, -77.0617568]]]

path_walk2: [[[38.8578163, -77.0595534], [38.8578163, -77.0595534]]]

total_time: 1577



## 2. google suggestion (metro + bus + walking)

In [170]:
def time_of_googlemap(depart,dest,depart_time='2019-04-13-09-30'):
    '''
    it's a problem of walking + bus(optional) + metro + bus(optional) + walking
    Args : depart --> (latitude,longitude)
           dest --> (latitude,longitude)
           departure time
    Return : path_walk, path_transit, total_time
    '''
    result = g.travel(origin=depart,
                      destination=dest, 
                      mode='transit',
                      departure_time=depart_time)
    total_time = result['total_time']
    # path
    path = []
    
    for i in result['steps']:
        tmp_depart = []
        tmp_depart.append(i['start']['lat'])
        tmp_depart.append(i['start']['lng'])

        tmp_dest = []
        tmp_dest.append(i['end']['lat'])
        tmp_dest.append(i['end']['lng'])

        path.append([tmp_depart,tmp_dest])

    return path, total_time

In [171]:
path,total_time = time_of_googlemap(depart,dest)
print('path:',path)
print()
print('total_time:',total_time)

path: [[[38.8860017, -77.0492223], [38.8843884, -77.06365319999999]], [[38.8845641, -77.0631731], [38.862692, -77.0595202]], [[38.8626772, -77.0596499], [38.8578225, -77.0595745]]]

total_time: 2037


## 3. walk + bicycle + metro 

In [167]:
# a basic version
# walk + bicycle + metro + walk
def walk_bicycle_metro(depart,dest,depart_time='2019-04-13-09-30'):
    '''
    it's a problem of walking + bicycle + metro
    Args : depart --> (latitude,longitude)
           dest --> (latitude,longitude)
    Return : path_walk,path_bicycle,path_metro,total_time
    '''
    bike_depart,bike_dest,bike_depart_name,bike_dest_name = find_bike.find_neareast_bike_station(depart, dest)

    walk_result = g.travel(origin=depart,
                           destination=bike_depart[0], 
                           mode='walking',
                           departure_time=depart_time)
    walk_time = walk_result['total_time']
    depart_time2 = add_time(depart_time,walk_time)
    
    #path walk
    path_walk = []
    for i in walk_result['steps']:
        if i['travel_mode'] == 'WALKING':
            tmp_depart = []
            tmp_depart.append(i['start']['lat'])
            tmp_depart.append(i['start']['lng'])

            tmp_dest = []
            tmp_dest.append(i['end']['lat'])
            tmp_dest.append(i['end']['lng'])

            path_walk.append([tmp_depart,tmp_dest])

    bicycle_result = g.travel(origin=bike_depart[0],
                              destination=bike_dest[0], 
                              mode='bicycling',
                              departure_time=depart_time2)

    bicycle_time = bicycle_result['total_time']
    depart_time3 = add_time(depart_time2,bicycle_time)
    
    #path bicycle
    path_bicycle = []
    for i in bicycle_result['steps']:

        tmp_depart = []
        tmp_depart.append(i['start']['lat'])
        tmp_depart.append(i['start']['lng'])

        tmp_dest = []
        tmp_dest.append(i['end']['lat'])
        tmp_dest.append(i['end']['lng'])

        path_bicycle.append([tmp_depart,tmp_dest])

    metro_walk_result = g.travel(origin=bike_dest[0],
                                 destination=dest, 
                                 mode='transit',
                                 departure_time=depart_time3)
    metro_walk_time = metro_walk_result['total_time']
    path_metro = []
    path_walk2 = []
    for i in result['steps']:
        if i['travel_mode'] == 'WALKING':
            tmp_depart = []
            tmp_depart.append(i['start']['lat'])
            tmp_depart.append(i['start']['lng'])

            tmp_dest = []
            tmp_dest.append(i['end']['lat'])
            tmp_dest.append(i['end']['lng'])

            path_walk2.append([tmp_depart,tmp_dest])

        if i['travel_mode'] == 'TRANSIT':
            tmp_depart2 = []
            tmp_depart2.append(i['start']['lat'])
            tmp_depart2.append(i['start']['lng'])

            tmp_dest2 = []
            tmp_dest2.append(i['end']['lat'])
            tmp_dest2.append(i['end']['lng'])

            path_metro.append([tmp_depart2,tmp_dest2]) 

    total_time = walk_time + bicycle_time + metro_walk_time
    return path_walk,path_bicycle,path_metro,path_walk2,total_time

In [169]:
path_walk,path_bicycle,path_metro,path_walk2, total_time = walk_bicycle_metro(depart,dest)
print('path_walk:',path_walk)
print()
print('path_bicycle:',path_bicycle)
print()
print('path_metro:',path_metro)
print()
print('path_walk2:',path_walk2)
print()
print('total_time:',total_time)

path_walk: [[[38.8860017, -77.0492223], [38.8860017, -77.0492223]]]

path_bicycle: [[[38.8860017, -77.0492223], [38.8870097, -77.04997949999999]], [[38.8870097, -77.04997949999999], [38.8869307, -77.03761589999999]], [[38.8869307, -77.03761589999999], [38.8868707, -77.03762610000001]], [[38.8868707, -77.03762610000001], [38.8838731, -77.0331922]], [[38.8838731, -77.0331922], [38.8799921, -77.0353059]], [[38.8799921, -77.0353059], [38.879887, -77.0373224]], [[38.879887, -77.0373224], [38.87464370000001, -77.0454924]], [[38.87464370000001, -77.0454924], [38.8607933, -77.0452198]], [[38.8607933, -77.0452198], [38.8565309, -77.0466082]], [[38.8565309, -77.0466082], [38.8581182, -77.0492113]], [[38.8581182, -77.0492113], [38.8576068, -77.049249]], [[38.8576068, -77.049249], [38.8573781, -77.0557524]], [[38.8573781, -77.0557524], [38.8564113, -77.0556182]], [[38.8564113, -77.0556182], [38.8563718, -77.05763429999999]], [[38.8563718, -77.05763429999999], [38.8572073, -77.057667]], [[38.857207

## 4. Comparison and pick one best methods among these 3

In [165]:
def comparison():
    ff, hh, gg, t1 = time_of_bicycling(depart,dest)
    dd, ee,t2 = time_of_googlemap(depart,dest)
    aa,bb,cc,t3 = walk_bicycle_metro(depart,dest)
    min_time = min(t1,t2,t3)
    if t1 == min_time:
        print('best solution: only use bicycle.')
    if t2 == min_time:
        print('best solution: use google map solution.')
    if t3 == min_time:
        print('best solution: use both bicycle and metro.')

In [166]:
comparison()

best solution: only use bicycle.
best solution: use both bicycle and metro.
